# Reglas de asociación y patrones secuenciales
---
Práctica de laboratorio 2 - Convocatoria Extraordinaria

**Grupo A08**

**Fecha de entrega:** 20/06/2024, 23:59

---
---

# Práctica 1: Reglas de Asociación

El objetivo del ejercicio es usar la librería `mlxtend` que nos permita solucionar todos los problemas relacionados con las reglas de
asociación. Para ello tendremos que usar el algoritmo Apriori y una serie de métodos para obtener la distinta
información que este genera.

Para ello se limpiarán los datos, se transformarán y finalmente se aplicará el algoritmo Apriori para analizar los resultados obtenidos.

Antes de comenzar se descargará el Dataset `Market_Basket_Optimisation.csv` proporcionado y se instalarán e importarán las librerías a usar.

In [47]:
pip install mlxtend

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [49]:
data = pd.read_csv('Market_Basket_Optimisation.csv', header=None)

data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Limpieza de los datos

En este caso, aparentemente se ven diversos valores NaN, aun así al tratarse cada fila de un registro de la compra, no todo el mundo compra 20 productos cuando va, además no hay una columna específica para cada producto, por lo tanto no se deben eliminar aquellas filas con valores NaN.

En general, para esta base de datos y debido al problema que se está tratando de resolver no es necesaria la limpieza de los datos, por tanto, se pasará directamente a la transformación de los mismos.

## Transformación de los datos

Al tratarse de un ejercicio de Reglas de Asociación, en concreto, un problema de optimización de la lista de la compra, se debe trabajar con listas de distinto tamaño así que hay que cambiar el formato de los datos a uno que la librería entienda.

In [50]:
transactions = data.applymap(lambda x: str(x)).values.tolist()
transactions = [[item for item in transaction if item != 'nan'] for transaction in transactions]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


A continuación, como el algoritmo apriori trabaja con variables binarias, se empleará la función `TransactionEncoder` que transformará la lista en una matriz binaria de manera que cada fila es una transacción y cada columna aporta información acerca de si está presente (1) o no (0) cada item.

In [51]:
te = TransactionEncoder()
matriz = te.fit(transactions).transform(transactions)
datos = pd.DataFrame(matriz, columns=te.columns_)

datos.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,False,True,True,False,True,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


Una vez que se obtienen los datos en el formato correspondiente, se aplicará el algoritmo Apriori.

## Aplicación del Algoritmo Apriori

### Cambiando el soporte mínimo

Para estos casos se usará la `métrica lift` y una `frecuencia de 1`.

En primer lugar se usará un `soporte mínimo de 0.01`.

In [52]:
s_freq1 = apriori(datos, min_support=0.01,use_colnames=True)
s_reglas1 = association_rules(s_freq1, metric='lift', min_threshold=1)
s_freq1

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.020397,(almonds)
1,0.033329,(avocado)
2,0.010799,(barbecue sauce)
3,0.014265,(black tea)
4,0.011465,(body spray)
...,...,...
252,0.011065,"(milk, ground beef, mineral water)"
253,0.017064,"(ground beef, mineral water, spaghetti)"
254,0.015731,"(milk, mineral water, spaghetti)"
255,0.010265,"(olive oil, mineral water, spaghetti)"


In [53]:
print(s_reglas1.shape)
s_reglas1.sort_values(by='confidence', ascending=False).head()

(406, 10)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
343,"(ground beef, eggs)",(mineral water),0.019997,0.238368,0.010132,0.506667,2.125563,0.005365,1.543848,0.540342
376,"(milk, ground beef)",(mineral water),0.021997,0.238368,0.011065,0.503030,2.110308,0.005822,1.532552,0.537969
319,"(ground beef, chocolate)",(mineral water),0.023064,0.238368,0.010932,0.473988,1.988472,0.005434,1.447937,0.508837
364,"(milk, frozen vegetables)",(mineral water),0.023597,0.238368,0.011065,0.468927,1.967236,0.005440,1.434136,0.503555
273,(soup),(mineral water),0.050527,0.238368,0.023064,0.456464,1.914955,0.011020,1.401255,0.503221


A continuación se usará un `soporte mínimo de 0.05`.

In [54]:
s_freq2 = apriori(datos, min_support=0.05,use_colnames=True)
s_reglas2 = association_rules(s_freq2, metric='lift', min_threshold=1)
s_freq2

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.087188,(burgers)
1,0.081056,(cake)
2,0.059992,(chicken)
3,0.163845,(chocolate)
4,0.080389,(cookies)
5,0.051060,(cooking oil)
6,0.179709,(eggs)
7,0.079323,(escalope)
8,0.170911,(french fries)
9,0.063325,(frozen smoothie)


In [55]:
print(s_reglas2.shape)
s_reglas2.sort_values(by='confidence', ascending=False)

(6, 10)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
5,(spaghetti),(mineral water),0.174110,0.238368,0.059725,0.343032,1.439085,0.018223,1.159314,0.369437
1,(chocolate),(mineral water),0.163845,0.238368,0.052660,0.321400,1.348332,0.013604,1.122357,0.308965
3,(eggs),(mineral water),0.179709,0.238368,0.050927,0.283383,1.188845,0.008090,1.062815,0.193648
4,(mineral water),(spaghetti),0.238368,0.174110,0.059725,0.250559,1.439085,0.018223,1.102008,0.400606
0,(mineral water),(chocolate),0.238368,0.163845,0.052660,0.220917,1.348332,0.013604,1.073256,0.339197
2,(mineral water),(eggs),0.238368,0.179709,0.050927,0.213647,1.188845,0.008090,1.043158,0.208562


Finalmente, se empleará un `soporte mínimo de 0.1`.

In [56]:
s_freq3 = apriori(datos, min_support=0.1,use_colnames=True)
s_reglas3 = association_rules(s_freq3, metric='lift', min_threshold=1)
s_freq3

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.163845,(chocolate)
1,0.179709,(eggs)
2,0.170911,(french fries)
3,0.132116,(green tea)
4,0.129583,(milk)
5,0.238368,(mineral water)
6,0.174110,(spaghetti)


In [57]:
print(s_reglas3.shape)
s_reglas3.sort_values(by='confidence', ascending=False).head()

(0, 10)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric


#### Conclusiones

Cambiando el soporte mínimo observamos que el número de ítems frecuentes varía. Cuanto mayor es el soporte, menor es el número de esos ítemsets frecuentes. Además, cuanto menor sea ese número, menos reglas serán generadas.

Cuando el `soporte mínimo era igual a 0.01`, se generaron `257 ítemsets` frecuentes, llegando a k = 3. Se generaron `406 reglas`, algunas son:

*   Si compra carne picada y huevos, compra agua mineral.
*   Si compra carne picada y leche, compra agua mineral.
*   Si compra carne picada y chocolate, compra agua mineral.
*   Si compra verduras congeladas y leche, compra agua mineral.
*   Si compra sopa, compra agua mineral.

Todas estas reglas tienen una confianza entre el 45% y 51%.


Por otro lado, cuando el `soporte mínimo era igual a 0.05`, se generaron `28 ítemsets` frecuentes, llegando a k = 2. Se generaron `6 reglas`, algunas son:

*   Si compra espaguetis, compra agua mineral y viceversa.
*   Si compra chocolate, compra agua mineral y viceversa.
*   Si compra huevos, compra agua mineral y viceversa.


 Todas con una confianza entre el 20% y 35% siendo menor que en el caso anterior.

 Finalmente, cuando el `soporte mínimo era igual a 0.1`, se generaron `7 ítemsets` frecuentes, llegando únicamente a k = 1. Además, no se generaron reglas, por lo tanto se descarta esta configuración.


 Aunque una mayor cantidad de reglas puede llegar a formar reglas inútiles, cuando el soporte era 0.01, se generaron reglas con mayor confianza y lift, por lo tanto se llega a la conclusión de que es la mejor configuración de soporte.

 A continuación se verá qué ocurre si se cambia la frecuencia.

### Cambiando la frecuencia

Como se ha observado con anterioridad el mejor `soporte mínimo es de 0.1`, por lo tanto, se mantendrá ese valor para estos nuevos casos, empleando de nuevo la `métrica lift`.

Se probará con una `frecuencia de 0.5`.

In [58]:
f_freq1 = apriori(datos, min_support=0.01,use_colnames=True)
f_reglas1 = association_rules(f_freq1, metric='lift', min_threshold=0.5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [59]:
print(f_reglas1.shape)
f_reglas1.sort_values(by='confidence', ascending=False).head()

(432, 10)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
367,"(ground beef, eggs)",(mineral water),0.019997,0.238368,0.010132,0.506667,2.125563,0.005365,1.543848,0.540342
402,"(milk, ground beef)",(mineral water),0.021997,0.238368,0.011065,0.503030,2.110308,0.005822,1.532552,0.537969
343,"(ground beef, chocolate)",(mineral water),0.023064,0.238368,0.010932,0.473988,1.988472,0.005434,1.447937,0.508837
390,"(milk, frozen vegetables)",(mineral water),0.023597,0.238368,0.011065,0.468927,1.967236,0.005440,1.434136,0.503555
297,(soup),(mineral water),0.050527,0.238368,0.023064,0.456464,1.914955,0.011020,1.401255,0.503221


Se probará con una `frecuencia de 1.2`.

In [60]:
f_freq2 = apriori(datos, min_support=0.01,use_colnames=True)
f_reglas2 = association_rules(f_freq2, metric='lift', min_threshold=1.2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [61]:
print(f_reglas2.shape)
f_reglas2.sort_values(by='confidence', ascending=False).head()

(348, 10)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
285,"(ground beef, eggs)",(mineral water),0.019997,0.238368,0.010132,0.506667,2.125563,0.005365,1.543848,0.540342
318,"(milk, ground beef)",(mineral water),0.021997,0.238368,0.011065,0.503030,2.110308,0.005822,1.532552,0.537969
261,"(ground beef, chocolate)",(mineral water),0.023064,0.238368,0.010932,0.473988,1.988472,0.005434,1.447937,0.508837
306,"(milk, frozen vegetables)",(mineral water),0.023597,0.238368,0.011065,0.468927,1.967236,0.005440,1.434136,0.503555
215,(soup),(mineral water),0.050527,0.238368,0.023064,0.456464,1.914955,0.011020,1.401255,0.503221


Se probará con una `frecuencia de 0.3`.

In [62]:
f_freq3 = apriori(datos, min_support=0.01,use_colnames=True)
f_reglas3 = association_rules(f_freq3, metric='lift', min_threshold=0.3)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [63]:
print(f_reglas3.shape)
f_reglas3.sort_values(by='confidence', ascending=False).head()

(432, 10)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
367,"(ground beef, eggs)",(mineral water),0.019997,0.238368,0.010132,0.506667,2.125563,0.005365,1.543848,0.540342
402,"(milk, ground beef)",(mineral water),0.021997,0.238368,0.011065,0.503030,2.110308,0.005822,1.532552,0.537969
343,"(ground beef, chocolate)",(mineral water),0.023064,0.238368,0.010932,0.473988,1.988472,0.005434,1.447937,0.508837
390,"(milk, frozen vegetables)",(mineral water),0.023597,0.238368,0.011065,0.468927,1.967236,0.005440,1.434136,0.503555
297,(soup),(mineral water),0.050527,0.238368,0.023064,0.456464,1.914955,0.011020,1.401255,0.503221


#### Conclusiones

Tras probar las distintas configuraciones se observa que cuando `min_threshold era igual a 0.5` se generaron 432 reglas más que cuando `min_threshold era igual a 1`. Las reglas generadas tienen en ambos casos la misma confianza, además de ser (las que más confianza tienen), las mismas reglas.

*   Si compra carne picada y huevos, compra agua mineral.
*   Si compra carne picada y leche, compra agua mineral.
*   Si compra carne picada y chocolate, compra agua mineral.
*   Si compra verduras congeladas y leche, compra agua mineral.
*   Si compra sopa, compra agua mineral.

Por otro lado, cuando `min_threshold era igual a 1.2` se generaron 348 reglas.
Por último,  cuando `min_threshold era igual a 0.3`, se generaron 432, al igual que en el primer caso.

Con esto se llega a la conclusión de que cuanto mayor sea la frecuencia, menos reglas se generarán, sin embargo, esto se cumple a partir de un valor, en este caso `0.5`.

Además, como hay que buscar el equilibrio entre la cantidad de reglas y hay que obtar por aquellas con mayor confianza y lift, al ser estos valores iguales en todos los casos, se llega a la conclusión de que la mejor configuración de cuando `min_threshold era igual a 1.2`, pues genera menos reglas que el resto (sin ser esta cantidad extremadamente pequeña), siendo igual de útiles.

### Cambiando la métrica

Finalmente, dejando el `soporte mínimo de 0.01` y una `frecuencia de 1. 2`, se probarán distintas métricas hasta llegar a la mejor configuración.

Para la ` métrica confidence` que mide la probabilidad de que un objeto se encuentre en una transacción que ya contiene a otro objeto:

In [64]:
m_freq1 = apriori(datos, min_support=0.01,use_colnames=True)
m_reglas1 = association_rules(m_freq1, metric='confidence', min_threshold=1.2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [65]:
print(m_reglas1.shape)
m_reglas1.sort_values(by='confidence', ascending=False).head()

(0, 10)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric


Para la ` métrica leverage` que mide la diferencia entre la frecuencia observada de la coocurrencia de dos ítems y la frecuencia esperada si fueran independientes:

In [66]:
m_freq2 = apriori(datos, min_support=0.01,use_colnames=True)
m_reglas2 = association_rules(m_freq2, metric='leverage', min_threshold=1.2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [67]:
print(m_reglas2.shape)
m_reglas2.sort_values(by='leverage', ascending=False).head()

(0, 10)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric


Para la ` métrica conviction` que mide la dependencia de la regla, es decir, evalúa cuántas veces se espera que ocurra un evento sin que pase otro:

In [68]:
m_freq3 = apriori(datos, min_support=0.01,use_colnames=True)
m_reglas3 = association_rules(m_freq3, metric='conviction', min_threshold=1.2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [69]:
print(m_reglas3.shape)
m_reglas3.sort_values(by='confidence', ascending=False).head()

(45, 10)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
27,"(ground beef, eggs)",(mineral water),0.019997,0.238368,0.010132,0.506667,2.125563,0.005365,1.543848,0.540342
35,"(milk, ground beef)",(mineral water),0.021997,0.238368,0.011065,0.503030,2.110308,0.005822,1.532552,0.537969
21,"(ground beef, chocolate)",(mineral water),0.023064,0.238368,0.010932,0.473988,1.988472,0.005434,1.447937,0.508837
31,"(milk, frozen vegetables)",(mineral water),0.023597,0.238368,0.011065,0.468927,1.967236,0.005440,1.434136,0.503555
15,(soup),(mineral water),0.050527,0.238368,0.023064,0.456464,1.914955,0.011020,1.401255,0.503221


#### Conclusiones

Mientras que las métricas `confidence` y `leverage` no generan ninguna regla, se observa que la` métrica conviction` genera 45 reglas siendo considerablemente menor que si la métrica es `lift`, sin embargo, en ambas coinciden las principales reglas:

*   Si compra carne picada y huevos, compra agua mineral.
*   Si compra carne picada y leche, compra agua mineral.
*   Si compra carne picada y chocolate, compra agua mineral.
*   Si compra verduras congeladas y leche, compra agua mineral.
*   Si compra sopa, compra agua mineral.

Por tanto, se concluye que, para este caso la mejor configuración es de un `soporte mínimo de 0.01`, una `frecuencia de 1. 2` y la `métrica conviction` ya que se obtienen las reglas necesarias con una mayor confianza y lift.

Finalmente, se observa que las reglas que se repiten en todas las configuraciones son:

*   Si compra carne picada y huevos, compra agua mineral.
*   Si compra carne picada y leche, compra agua mineral.
*   Si compra carne picada y chocolate, compra agua mineral.
*   Si compra verduras congeladas y leche, compra agua mineral.
*   Si compra sopa, compra agua mineral.

Todas estas reglas tienen una confianza entre el 45% y 51%. Además todas tienen en común el agua mineral, por lo que se puede deducir que la mayoría de los clientes, cuando van a comprar, suelen comprar agua mineral, siendo una posible idea colocar el puesto de agua mineral cerca de la caja registradora del supermercado.

### **¿Qué diferencias hay entre usar soporte y frecuencia?**

En la práctica, se puede decir que manteniendo una frecuencia y métrica constantes, al disminuir el soporte mínimo aumentaban los ítems más frecuentes y con ellos las reglas (cuando el soporte mínimo era 0.01 se generaron 406 reglas mientras que cuando el soporte mínimo era 0.05 se generaron 6 reglas).

Por otro lado, lo mismo ocurre con la fracuencia, cuanta mayor frecuencia, menos reglas (cuando la frecuencia era de 0.5 se generaron 432 reglas mientras que cuando era 1.2 se generaron 348).

Sin embargo, sí se puede decir que al cambiar el soporte mínimo ligeramente, el número de reglas se reducía considerablemente (de 0.01 a 0.05 se han reducido 400 reglas) mientras que al cambiar significativamente la frecuencia, la diferencia de reglas generadas no era tan notoria (de 0.3 a 1.2 se han reducido 84 reglas). Por esto se podría deducir que el soporte mínimo tiene más peso que la frecuencia.

Además, la frecuencia de un conjunto de ítems se refiere al número total de veces que aparece ese conjunto en la base de datos, mientras que el soporte de un conjunto de ítems (o de una regla de asociación) se refiere a la proporción de transacciones en la base de datos que contienen dicho conjunto de ítems.